In [ ]:
import json

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import piexif
import os
from datetime import datetime
from csv import DictWriter

from pathlib import Path
from fnmatch import fnmatch
from pymavlink import mavutil
from inspect import getfullargspec
from datetime import datetime
from pixhawk_telelemtry_import import Telemetry

In [ ]:
# ROV .tlog file
tlog_file_name = r"/Users/matthewtoberman/Downloads/2024-03-02 10-24-00.tlog"
#  read in WL
ugps_df = pd.read_csv(r"/Users/matthewtoberman/Downloads/WL_global_positions_20240302-092413.csv")


In [ ]:
# Waterlinked
# create datetime and assign this to index
# format heading into decimal degrees
def format_ugps_time(timestr):
    return datetime.strptime(timestr[0:4] + '-' + timestr[4:6] + '-' + timestr[6:8] + ' ' + timestr[9:11] + ':' + timestr[11:13] + ':' + timestr[13:15], '%Y-%m-%d %H:%M:%S')

ugps_df['date_time'] = ugps_df.apply(lambda row : format_ugps_time(row['time']), axis = 1)
ugps_df.index = ugps_df['date_time']
del ugps_df['date_time']
ugps_df.sort_index(inplace=True, ascending=True)
ugps_df = ugps_df.resample('1S').mean()
ugps_df= ugps_df.interpolate(method ='linear', limit_direction ='backward', limit = 10).copy()

# Agisoft 
# agisoft_pos_df = pd.read_csv("/Users/matthewtoberman/Dropbox/TRITONIA/ROV_SENSOR_FUSION/broad_bay_testing/T_4A_EstimatePos.csv",skiprows=1)

# ROV
ROV_CLOCK_HOUR_ADJUST=0
if tlog_file_name.split('/')[-1].split('.')[0]+'.csv' not in os.listdir('/'.join(tlog_file_name.split('/')[0:-1])):
    combined_CSV_output_filename = None
    Telemetry.logs_to_csv(combined_CSV_output_filename, [tlog_file_name],Telemetry.DEFAULT_FIELDS)
    rov_log_df = Telemetry.csv_to_df(tlog_file_name[0:-4]+'csv')
    rov_log_df.index = rov_log_df.index + pd.DateOffset(hours=ROV_CLOCK_HOUR_ADJUST)

rov_log_df = Telemetry.csv_to_df(tlog_file_name[0:-4]+'csv')


#  remove strange australian time zone
if rov_log_df.index[1].tzinfo != 'none':
    rov_log_df.index = rov_log_df.index.tz_convert(None)




rov_log_df['lat_test'] =  [len(str(decimal)) for decimal in  list(rov_log_df['GLOBAL_POSITION_INT.lat'])]

rov_log_valid_positions_df  = rov_log_df[rov_log_df['lat_test'] > 3]

# create python date time
# rov_log_valid_positions_df ['date_time']=pd.to_datetime(rov_log_valid_positions_df ['SYSTEM_TIME.time_unix_usec'], unit='us', origin='unix')

# format lat lon into decimal degrees
def format_hdg(latlon):
    return int(str(latlon)[:-7])+float((str(latlon)[-7:]))/10000000
rov_log_valid_positions_df['formatted_lat'] = rov_log_valid_positions_df.apply(lambda row : format_hdg(row['GLOBAL_POSITION_INT.lat']), axis = 1)
rov_log_valid_positions_df['formatted_lon'] = rov_log_valid_positions_df.apply(lambda row : format_hdg(row['GLOBAL_POSITION_INT.lon']), axis = 1)

# interpolate using mean pixhawk data to 1 hz matchting UGPS
mav_df_filtered_1s = rov_log_valid_positions_df.copy()
mav_df_filtered_1s.sort_index(inplace=True, ascending=True)
mav_df_filtered_1s = mav_df_filtered_1s.resample('1S').mean()
mav_df_filtered_1s= mav_df_filtered_1s.interpolate(method ='linear', limit_direction ='backward', limit = 10).copy()
# crop to only get times that also have ugps values
mav_df_filtered_1s_crop = mav_df_filtered_1s.loc[(ugps_df.index.get_level_values(0)[0]):(ugps_df.index.get_level_values(0)[-1])].copy()

print(mav_df_filtered_1s.index[10])
print(f'UGPS time span : {ugps_df.index[0]} -  {ugps_df.index[-1]}')
print(f'Pixhawk time span : {mav_df_filtered_1s.index[0]} -  {mav_df_filtered_1s.index[-1]}')
print(f'Pixhawk and UGPS both available time span : {mav_df_filtered_1s_crop.index[0]} -  {mav_df_filtered_1s_crop.index[-1]}')

In [ ]:

#lon lat plot
plt.figure(figsize=(10, 10))
plt.scatter(mav_df_filtered_1s_crop['GLOBAL_POSITION_INT.lon']/10000000,mav_df_filtered_1s_crop['GLOBAL_POSITION_INT.lat']/10000000, s=4,c='red')
# plt.scatter(agisoft_pos_df['X_est']-0.0001,agisoft_pos_df[ 'Y_est']+0.0001, s=4,c='magenta' )
# plt.scatter(agisoft_pos_df['X/Longitude'],agisoft_pos_df[ 'Y/Latitude'], s=4,c='blue' )
plt.scatter(ugps_df['longitude'], ugps_df['latitude'], s=4,c='cyan')
f = 1.0/np.cos(56.4*np.pi/180)
plt.gca().set_aspect(f)
plt.ticklabel_format(useOffset=False)
plt.gca().ticklabel_format(style='plain')
# plt.xlim(-6.284,-6.283) 
# plt.ylim(58.2320,58.2326) 
plt.xlabel('Longitude') 
plt.ylabel('Latitude') 
plt.legend(["global_pos_int", "WL UGPS"], loc="lower right")


# depth plot
plt.figure(figsize=(15, 15))
plt.scatter(mav_df_filtered_1s_crop.index,mav_df_filtered_1s_crop['VFR_HUD.alt'], s=4,c='red')
plt.scatter(ugps_df.index, ugps_df['altitude']*(-1), s=4,c='cyan')
plt.legend(["global_pos_int", "WL UGPS"], loc="lower right")




